In [ ]:
import torch
import numpy as np
import cv2
from model.u2net import U2NET

def apply_mask_and_save(image_path, output_path):
    model = U2NET(3, 1)
    model.load_state_dict(torch.load('/media/ubuntu/sda/data/trigger/nat_image_detection/u2net.pth'))
    model.eval()

    gray_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if gray_img is None:
        print(f"Error: Cannot read image {image_path}")
        return
    
    img_rgb = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img_rgb, (320, 320))
    img = img / 255.0
    img_tensor = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32).unsqueeze(0)

    with torch.no_grad():
        saliency_map = model(img_tensor)[0].squeeze().numpy()
    
    _, mask = cv2.threshold((saliency_map * 255).astype(np.uint8), 0, 255, cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    final_mask = None
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        center_x, center_y = x + w//2, y + h//2
        if (0.25*320 < center_x < 0.75*320) and (0.25*320 < center_y < 0.75*320):
            final_mask = cv2.drawContours(np.zeros_like(mask), [largest_contour], -1, 255, -1)
            final_mask = cv2.resize(final_mask, (gray_img.shape[1], gray_img.shape[0]))

    if final_mask is not None:
        vis_img = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)
        
        masked_img = cv2.bitwise_and(vis_img, vis_img, mask=final_mask)
        
        # 方法2：提取前景为透明背景（PNG格式）
        # masked_img = cv2.cvtColor(vis_img, cv2.COLOR_BGR2BGRA)
        # masked_img[:, :, 3] = final_mask * 255
        
        cv2.imwrite(output_path, masked_img)
        print(f"Masked image saved to {output_path}")
    else:
        # 保存原图作为兜底
        cv2.imwrite(output_path, gray_img)
        print(f"No mask found, original image saved to {output_path}")

# 使用示例
apply_mask_and_save("/media/ubuntu/sda/data/trigger/NaturalImages_new_2/12.jpg", "../output_masked.jpg")

Masked image saved to ../output_masked.jpg


In [2]:
import torch
import numpy as np
import cv2
from model.u2net import U2NET

def apply_mask_and_save(image_path, output_path):
    # 加载预训练模型
    model = U2NET(3, 1)
    model.load_state_dict(torch.load('/media/ubuntu/sda/data/trigger/nat_image_detection/u2net.pth'))
    model.eval()

    # 读取并预处理图像
    gray_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if gray_img is None:
        print(f"Error: Cannot read image {image_path}")
        return
    
    # 生成显著图掩膜
    img_rgb = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img_rgb, (320, 320))
    img = img / 255.0
    img_tensor = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32).unsqueeze(0)

    with torch.no_grad():
        saliency_map = model(img_tensor)[0].squeeze().numpy()
    
    # 二值化处理
    _, mask = cv2.threshold((saliency_map * 255).astype(np.uint8), 0, 255, cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    final_mask = None
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        final_mask = cv2.drawContours(np.zeros_like(mask), [largest_contour], -1, 255, -1)
        final_mask = cv2.resize(final_mask, (gray_img.shape[1], gray_img.shape[0]))

    # 应用掩膜生成图像
    if final_mask is not None:
        # 将灰度原图转为三通道便于可视化
        vis_img = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)
        
        # 将图像转换为 BGRA 格式（添加透明通道）
        masked_img = cv2.cvtColor(vis_img, cv2.COLOR_BGR2BGRA)
        
        # 将透明通道的值设置为掩膜的值
        masked_img[:, :, 3] = final_mask
        
        # 保存为 PNG 格式，以保留透明通道
        cv2.imwrite(output_path, masked_img)
        print(f"Masked image with transparent background saved to {output_path}")
    else:
        # 保存原图作为兜底
        cv2.imwrite(output_path, gray_img)
        print(f"No mask found, original image saved to {output_path}")



In [3]:
apply_mask_and_save(f"/media/ubuntu/sda/data/trigger/NaturalImages_new_2/12.jpg", f"/media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask/12.png")

/media/ubuntu/sda/data/trigger/nat_image_detection/U-2-Net-master/model/u2net.py:23: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  src = F.upsample(src,size=tar.shape[2:],mode='bilinear')


Masked image with transparent background saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask/12.png


In [25]:
import os
image_dir = os.listdir("/media/ubuntu/sda/data/trigger/NaturalImages_new_2/")
for image in image_dir:
    image = image.split(".")[0]
    apply_mask_and_save(f"/media/ubuntu/sda/data/trigger/NaturalImages_new_2/{image}.jpg", f"/media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask/{image}.png")


Masked image with transparent background saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask/108.png
Masked image with transparent background saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask/11.png
Masked image with transparent background saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask/24.png
Masked image with transparent background saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask/47.png
Masked image with transparent background saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask/48.png
Masked image with transparent background saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask/40.png
Masked image with transparent background saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask/110.png
Masked image with transparent background saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask/115.png
Masked image 

In [4]:
import torch
import numpy as np
import cv2
from model.u2net import U2NET

def apply_mask_and_save(image_path, output_path):
    # 加载预训练模型
    model = U2NET(3, 1)
    model.load_state_dict(torch.load('/media/ubuntu/sda/data/trigger/nat_image_detection/u2net.pth'))
    model.eval()

    # 读取并预处理图像
    gray_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if gray_img is None:
        print(f"Error: Cannot read image {image_path}")
        return
    
    # 生成显著图掩膜
    img_rgb = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img_rgb, (320, 320))
    img = img / 255.0
    img_tensor = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32).unsqueeze(0)

    with torch.no_grad():
        saliency_map = model(img_tensor)[0].squeeze().numpy()
    
    # 二值化处理
    _, mask = cv2.threshold((saliency_map * 255).astype(np.uint8), 0, 255, cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    final_mask = None
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        center_x, center_y = x + w//2, y + h//2
        if (0.25*320 < center_x < 0.75*320) and (0.25*320 < center_y < 0.75*320):
            final_mask = cv2.drawContours(np.zeros_like(mask), [largest_contour], -1, 255, -1)
            final_mask = cv2.resize(final_mask, (gray_img.shape[1], gray_img.shape[0]))

    # 应用掩膜生成图像
    if final_mask is not None:
        # 将灰度原图转为三通道便于可视化
        vis_img = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)
        
        # 方法1：保留前景+背景置黑
        masked_img = cv2.bitwise_and(vis_img, vis_img, mask=final_mask)
        
        # 方法2：提取前景为透明背景（PNG格式）
        # masked_img = cv2.cvtColor(vis_img, cv2.COLOR_BGR2BGRA)
        # masked_img[:, :, 3] = final_mask * 255
        
        cv2.imwrite(output_path, masked_img)
        print(f"Masked image saved to {output_path}")
    else:
        # 保存原图作为兜底
        cv2.imwrite(output_path, gray_img)
        print(f"No mask found, original image saved to {output_path}")

In [5]:
import os
image_dir = os.listdir("/media/ubuntu/sda/data/trigger/NaturalImages_new_2/")
for image in image_dir:
    image = image.split(".")[0]
    apply_mask_and_save(f"/media/ubuntu/sda/data/trigger/NaturalImages_new_2/{image}.jpg", f"/media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask_black/{image}.png")

Masked image saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask_black/108.png
Masked image saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask_black/11.png
Masked image saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask_black/24.png
Masked image saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask_black/47.png
Masked image saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask_black/48.png
Masked image saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask_black/40.png
No mask found, original image saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask_black/110.png
Masked image saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask_black/115.png
Masked image saved to /media/ubuntu/sda/data/trigger/nat_image_detection/nat_image_mask_black/41.png
Masked image saved to /media/ubuntu/sda/data/trigger/nat_image_detectio